<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-данных" data-toc-modified-id="Описание-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание данных</a></span></li><li><span><a href="#Загрузка-библиотек" data-toc-modified-id="Загрузка-библиотек-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка библиотек</a></span></li><li><span><a href="#Загрузка-файла-и-осмотр-данных" data-toc-modified-id="Загрузка-файла-и-осмотр-данных-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Загрузка файла и осмотр данных</a></span></li><li><span><a href="#Подключаем-бота-и-пишем-первую-функцию" data-toc-modified-id="Подключаем-бота-и-пишем-первую-функцию-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Подключаем бота и пишем первую функцию</a></span><ul class="toc-item"><li><span><a href="#Напишем-первую-команду" data-toc-modified-id="Напишем-первую-команду-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Напишем первую команду</a></span></li><li><span><a href="#Добавляем-SQLite" data-toc-modified-id="Добавляем-SQLite-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Добавляем SQLite</a></span></li><li><span><a href="#Пишем-функции-нашему-боту" data-toc-modified-id="Пишем-функции-нашему-боту-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Пишем функции нашему боту</a></span><ul class="toc-item"><li><span><a href="#Добавим-продажи-в-список" data-toc-modified-id="Добавим-продажи-в-список-4.3.1"><span class="toc-item-num">4.3.1&nbsp;&nbsp;</span>Добавим продажи в список</a></span></li><li><span><a href="#Покажем-отчёт" data-toc-modified-id="Покажем-отчёт-4.3.2"><span class="toc-item-num">4.3.2&nbsp;&nbsp;</span>Покажем отчёт</a></span></li><li><span><a href="#Связываем-отчёт" data-toc-modified-id="Связываем-отчёт-4.3.3"><span class="toc-item-num">4.3.3&nbsp;&nbsp;</span>Связываем отчёт</a></span></li></ul></li></ul></li></ul></div>

# Проект : Написать телеграм бота

**Технические требования к разработке:**  
Наличие HTTP сервиса (Python), который имеет 2 метода:  
● добавить запись о продаже   
● получить данные о продажах за период  
**В телеграм боте должны быть доступны 2 кнопки:**  
● Внести данные о продаже  
● Получить отчет о продажах за период  
**Результат:**  
● ссылка на github  
● ссылка на бот  

## Описание данных

Таблица `nomenk_wine`  содержит сущности:  
- `date` - первый день месяца,  
- `lvl_5` - номенклатура,  
- `card_id` - ключ покупателя,   
- `checks` - количество уникальных чеков. 

## Загрузка библиотек

In [1]:
import pandas as pd
import requests
from urllib.parse import urlencode

## Загрузка файла и осмотр данных

In [2]:
# получаем прямую ссылку на файлы
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'


# ссылки на данные с указанием разделителя csv-файла
links_dic = {
    'nomenk_wine': ['https://disk.yandex.ru/d/A_mPari4YOl8ZA', ';']}

# скачиваем файлы
for csv_filename, csv_link in links_dic.items():
    final_url = base_url + urlencode(dict(public_key=csv_link[0]))
    response = requests.get(final_url)
    download_url = response.json()['href']
    globals()[csv_filename] = pd.read_csv(download_url, sep=csv_link[1])

Взглянем на данные:

In [3]:
nomenk_wine.head()

,date,lvl_5,card_id,checks
0,2022-01-01,ВИНО АРОМАТИЗИРОВАННОЕ,3338,1
1,2022-01-01,ВИНО АРОМАТИЗИРОВАННОЕ,3988,1
2,2022-01-01,ВИНО АРОМАТИЗИРОВАННОЕ,12237,1
3,2022-01-01,ВИНО АРОМАТИЗИРОВАННОЕ,14576,1
4,2022-01-01,ВИНО АРОМАТИЗИРОВАННОЕ,18316,1


In [4]:
nomenk_wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 474168 entries, 0 to 474167
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   date     474168 non-null  object
 1   lvl_5    474168 non-null  object
 2   card_id  474168 non-null  int64 
 3   checks   474168 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 14.5+ MB


Посмотрим на список товаров:

In [5]:
nomenk_wine.lvl_5.unique()

array(['ВИНО АРОМАТИЗИРОВАННОЕ', 'ВИНО БЕЛОЕ', 'ВИНО БЭГ-ИН-БОКС/ТЕТРАП.',
       'ВИНО КРАСНОЕ', 'ВИНО КРЕПЛЕНОЕ, КАГОР', 'ВИНО ПЛОДОВЫЕ',
       'ВИНО РОЗОВЫЕ', 'ВИСКИ', 'ВОДА 1Л И БОЛЕЕ', 'ВОДА МЕНЕЕ 1 ЛИТРА',
       'ВОДКА ОБЫКНОВЕННАЯ', 'ВОДКА ОСОБАЯ',
       'ГАЗИРОВАННЫЙ ВИННЫЙ НАПИТОК', 'ДИСТИЛЛЯТЫ ПРОЧИЕ',
       'ИМП. КОНЬЯК, БРЕНДИ, АРМАНЬЯК', 'КОНЬЯК, БРЕНДИ РОССИЯ',
       'ЛИКЕРЫ, БАЛЬЗАМЫ', 'НАПИТКИ Б/А 1Л И БОЛЕЕ',
       'НАПИТКИ Б/А МЕНЕЕ 1 Л', 'НАСТОЙКИ', 'ПИВО БОЛЕЕ 1 ЛИТРА',
       'ПИВО ДО 1 ЛИТРА', 'ПРЕДМЕТЫ ДЛЯ КУРЕНИЯ', 'РАЗЛИВНЫЕ НАПИТКИ',
       'СИГАРЕТЫ', 'СИРОПЫ И ФРУКТ. НАПИТКИ', 'СЛАБЫЙ АЛКОГОЛЬ',
       'СОКИ И НЕКТАРЫ 1 Л И МЕНЕЕ', 'СОКИ И НЕКТАРЫ В ИНД УПАКОВКЕ',
       'СОКИ И НЕКТАРЫ СВЫШЕ 1 Л.', 'ШАМПАНСКИЕ И ИГРИСТЫЕ'], dtype=object)

Сумма списка товаров:

In [6]:
nomenk_wine.lvl_5.nunique()

31

Посмотрим на общее количество покупателей:

In [7]:
nomenk_wine.card_id.count()

474168

Посмотрим на количество уникальных покупателей:

In [8]:
nomenk_wine.card_id.nunique()

68549

## Подключаем бота и пишем первую функцию

Для работы с API телеграма есть специальная библиотека – telebot. Нам нужно ее установить.

In [9]:
import telebot
from telebot import types

In [10]:
# подключим токен нашего бота
bot = telebot.TeleBot("5957665154:AAFuyapHxy_H_A6ALTJ9ZjKHPX5723MWALQ")

### Напишем первую команду

In [11]:
# напишем, что делать нашему боту при команде старт
@bot.message_handler(commands=['start']) #декоратор
def send_keyboard(message, text="Привет, чем я могу тебе помочь?"):
    keyboard = types.ReplyKeyboardMarkup(row_width=1)  # наша клавиатура
    itembtn1 = types.KeyboardButton('Добавить продажи') # создадим кнопку
    itembtn2 = types.KeyboardButton('Показать отчёт')
    keyboard.add(itembtn1, itembtn2) # добавим кнопки 1 и 2 на первый ряд
    # пришлем это все сообщением и запишем выбранный вариант
    msg = bot.send_message(message.from_user.id,
                     text=text, reply_markup=keyboard) #бот возращает юзеру клавиатуру с кнопками

    # отправим этот вариант в функцию, которая его обработает
    bot.register_next_step_handler(msg, callback_worker)

### Добавляем SQLite

Устанавливать ее не нужно – она встроена в питон. Можем сразу начинать работать. Подключим ее:

In [12]:
import sqlite3
import sqlalchemy

In [13]:
# подключаем базу данных
conn = sqlite3.connect('baza_nom.db')
# курсор для работы с таблицами
cursor = conn.cursor()

Загрузим наш датафрейм в базу данных:

In [14]:
cnx = sqlite3.connect('baza_nom.db')

In [15]:
#nomenk_wine.to_sql(name='planner2', con=cnx)

Выполним SQL запрос к нашей базе данных:

In [16]:
p2 = pd.read_sql('select lvl_5 from planner2', cnx)

In [17]:
p2

,lvl_5
0,ВИНО АРОМАТИЗИРОВАННОЕ
1,ВИНО АРОМАТИЗИРОВАННОЕ
2,ВИНО АРОМАТИЗИРОВАННОЕ
3,ВИНО АРОМАТИЗИРОВАННОЕ
4,ВИНО АРОМАТИЗИРОВАННОЕ
...,...
474164,ШАМПАНСКИЕ И ИГРИСТЫЕ
474165,ШАМПАНСКИЕ И ИГРИСТЫЕ
474166,ШАМПАНСКИЕ И ИГРИСТЫЕ
474167,ШАМПАНСКИЕ И ИГРИСТЫЕ


### Пишем функции нашему боту

В разделе 4.1  мы сделали 2 кнопки для нашего бота:

- Добавить продажи  
- Показать отчёт  

#### Добавим продажи в список

In [18]:
# напишем функции для каждого случая
# эта добавляет строчку с планом в хранилище
def add_plan(msg):
    with sqlite3.connect('baza_nom.db') as con:
        cursor = con.cursor()
        cursor.execute('INSERT INTO planner2 (card_id, lvl_5) VALUES (?, ?)',
                       (msg.from_user.id, msg.text))
        con.commit()
    bot.send_message(msg.chat.id, 'Запомню :-)')
    send_keyboard(msg, "Чем еще могу помочь?")

#### Покажем отчёт

In [19]:
# просто функция, которая делает нам красивые строки для отправки пользователю
def get_plans_string(tasks):
    tasks_str = []
    for val in list(enumerate(tasks)): # val=(0, (дело1))
        tasks_str.append(str(val[0] + 1) + ') ' + val[1][0] + '\n')
    return ''.join(tasks_str)

# отправляем пользователю его отчет
def show_plans(msg):
    with sqlite3.connect('baza_nom.db') as con:
        cursor = con.cursor()
        cursor.execute('SELECT count(card_id) FROM planner2'.format(msg.from_user.id))
        tasks = get_plans_string(cursor.fetchall())
        bot.send_message(msg.chat.id, tasks)
        send_keyboard(msg, "Чем еще могу помочь?")

#### Связываем отчёт

Отправим этот вариант в функцию, которая его обработает:

Напишем функцию, которая все обрабатывать:

In [20]:
# привязываем функции к кнопкам на клавиатуре
def callback_worker(call):
    if call.text == "Добавить продажи":
        msg = bot.send_message(call.chat.id, 'Давайте добавим данные!')
        bot.register_next_step_handler(msg, add_plan)

    elif call.text == "Показать отчёт":
        try:
            show_plans(call)
        except:
            bot.send_message(call.chat.id, 'Здесь пусто. :-)')
            send_keyboard(call, "Чем еще могу помочь?")

Допишем функцию, которая на любое текстовое сообщение будет заново кидать клавиатуру:

In [21]:
@bot.message_handler(content_types=['text'])
def handle_docs_audio(message):
    send_keyboard(message, text="Я не понимаю :-( Выберите один из пунктов меню:")

И последняя строчка:

In [ ]:
bot.polling(none_stop=True) # или bot.infinity_polling()

Это значит, что наш бот постоянно спрашивает у телеграма нет ли для него новых сообщений. Если мы запустимэту строчку, то Наш бот заработает! НО работать он будет только если Наш код запущен – то есть при выключении компьютера или закрытии программы бот отвечать перестанет.